In [77]:
using JuMP
using Ipopt
using Random
using Printf
using LinearAlgebra

Opening in existing browser session.
Opening in existing browser session.


In [78]:
## Function to solve a small instance of the deterministic model
function deterministic_model_small()

    ##################### PROBLEM DATA FOR SMALLER INSTANCE ########################
    Random.seed!(1)                   # Control random number generation
    nI = 20                           # Number of suppliers
    nJ = 30                           # Number of clients
    nS = 100                          # Number of scenarios
    I = 1:nI                          # Supplier index set
    J = 1:nJ                          # Client index set
    S = 1:nS                          # Scenario index set
    #### Generate random data for the problem
    c = rand(5:20, nI)                # Unit capacity costs of suppliers
    d = rand(nJ,nS).*rand(10:50, nJ)  # Client demands in all scenarios
    q = rand(5000:10000, nJ)          # Unit costs of unfulfilled demand
    p = ones(nS).*1/nS                # Scenario probabilities
    f = rand(3:45, (nI,nJ))           # Unit costs to fulfil demands
    b = rand(20:100, nI)              # Max supplier capacities
    B = 3000                          # Max budget (cost) for capacity acquisition
    ################################################################################
    
    ## We first solve the problem formulation directly without ADMM
    tini  = time()                                      # Start timer
    model = Model(with_optimizer(Ipopt.Optimizer))      # We use Ipopt solver to compare with ADMM   

    ## Variables
    @variable(model, x[i in I] >= 0)                    # Reserved capacity variables
    @variable(model, y[i in I, j in J, s in S] >= 0)    # Demand fulfilment variables
    @variable(model, u[j in J, s in S] >= 0)            # Unfulfilled demand variables

    ## Objective: Minimize the total phase 1 + phase 2 costs over all scenarios
    @objective(model, Min,
          sum(c[i]*x[i] for i in I) +
          sum(p[s]*f[i,j]*y[i,j,s] for s in S, i in I, j in J) + 
          sum(p[s]*q[j]*u[j,s] for s in S, j in J))

    ## Constraints
    @constraint(model, [i in I], x[i] <= b[i])          # Max capacity constraint
    @constraint(model, sum(c[i]*x[i] for i in I) <= B)  # Max capacity budget (cost) constraint

    ## Capacity reserve limit constraint for each supplier i in each scenario s
    @constraint(model, [i in I, s in S], sum(y[i,j,s] for j in J) <= x[i])

    ## Demand balance constraint (u[j,s] is unfulfilled demand of client j in scenario s)
    @constraint(model, DemBal[j in J, s in S], sum(y[i,j,s] for i in I) + u[j,s] == d[j,s])

    optimize!(model)                                    # Solve the problem

    status = termination_status(model)                  # Solution status
    println(status)                                     # Print status
    
    ## Stop timer
    tend = time() - tini
   
    ## Return solution vector x, costs c, and solution time tend
    return (c, x, tend)
    
end

deterministic_model_small (generic function with 1 method)

In [79]:
## NOTE: Run this cell twice, the first run is always slower due to "compilation" (Julia uses JIT compilation)
## Solve the small deterministic instance. 
(c, x, tend) = deterministic_model_small()

xsol = value.(x)                          # Get optimal x values (reserved capacities)
xsol = round.(xsol.data, digits = 2)      # Round to 2 decimals 
fval = dot(c, xsol)                       # Optimal cost of reserved capacities

## Solution time of small deterministic model
println("\nSolution time: ", tend, "\n")

## Print optimal solution 
println("Optimal solution:\n")
for i = 1:length(xsol)
    @printf("x[%2d] = %7.2f\n", i, xsol[i])
end

## Print optimal cost of reserved capacities
println("\nOptimal cost of reserved capacities: ", fval)

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:    63000
Number of nonzeros in inequality constraint Jacobian.:    62040
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:    63020
                     variables with only lower bounds:    63020
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:     3000
Total number of inequality constraints...............:     2021
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:     2021

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 


Solution time: 39.74964094161987

Optimal solution:

x[ 1] =    0.00
x[ 2] =   29.00
x[ 3] =    0.00
x[ 4] =   42.00
x[ 5] =   12.40
x[ 6] =    0.00
x[ 7] =   73.00
x[ 8] =    0.00
x[ 9] =   64.00
x[10] =   61.00
x[11] =    0.00
x[12] =   29.00
x[13] =   85.00
x[14] =    0.00
x[15] =    0.00
x[16] =    0.00
x[17] =    0.00
x[18] =    0.00
x[19] =   36.00
x[20] =    0.00

Optimal cost of reserved capacities: 3000.0


In [80]:
## Function to solve a large instance of the deterministic model
function deterministic_model_large()

    ##################### PROBLEM DATA FOR LARGER INSTANCE ########################
    Random.seed!(1)                   # Control random number generation
    nI = 20                           # Number of suppliers
    nJ = 30                           # Number of clients
    nS = 500                          # Number of scenarios
    I = 1:nI                          # Supplier index set
    J = 1:nJ                          # Client index set
    S = 1:nS                          # Scenario index set
    #### Generate random data for the problem
    c = rand(5:20, nI)                # Unit capacity costs of suppliers
    d = rand(nJ,nS).*rand(10:50, nJ)  # Client demands in all scenarios
    q = rand(5000:10000, nJ)          # Unit costs of unfulfilled demand
    p = ones(nS).*1/nS                # Scenario probabilities
    f = rand(3:45, (nI,nJ))           # Unit costs to fulfil demands
    b = rand(20:100, nI)              # Max supplier capacities
    B = 3000                          # Max budget (cost) for capacity acquisition
    ################################################################################
    
    ## We first solve the problem formulation directly without ADMM
    tini  = time()                                      # Start timer
    model = Model(with_optimizer(Ipopt.Optimizer))      # We use Ipopt solver to compare with ADMM   

    ## Variables
    @variable(model, x[i in I] >= 0)                    # Reserved capacity variables
    @variable(model, y[i in I, j in J, s in S] >= 0)    # Demand fulfilment variables
    @variable(model, u[j in J, s in S] >= 0)            # Unfulfilled demand variables

    ## Objective: Minimize the total phase 1 + phase 2 costs over all scenarios
    @objective(model, Min,
          sum(c[i]*x[i] for i in I) +
          sum(p[s]*f[i,j]*y[i,j,s] for s in S, i in I, j in J) + 
          sum(p[s]*q[j]*u[j,s] for s in S, j in J))

    ## Constraints
    @constraint(model, [i in I], x[i] <= b[i])          # Max capacity constraint
    @constraint(model, sum(c[i]*x[i] for i in I) <= B)  # Max capacity budget (cost) constraint

    ## Capacity reserve limit constraint for each supplier i in each scenario s
    @constraint(model, [i in I, s in S], sum(y[i,j,s] for j in J) <= x[i])

    ## Demand balance constraint (u[j,s] is unfulfilled demand of client j in scenario s)
    @constraint(model, DemBal[j in J, s in S], sum(y[i,j,s] for i in I) + u[j,s] == d[j,s])

    optimize!(model)                                    # Solve the problem

    status = termination_status(model)                  # Solution status
    println(status)                                     # Print status 
    
    ## Stop timer
    tend = time() - tini
    
    ## Return solution vector x, costs c, and solution time tend
    return (c, x, tend)
end

deterministic_model_large (generic function with 1 method)

In [81]:
## NOTE: Run this cell twice, the first run is always slower due to "compilation" (Julia uses JIT compilation)
## Solve the large deterministic model
(c, x, tend) = deterministic_model_large()

xsol = value.(x)                          # Get optimal x values (reserved capacities)
xsol = round.(xsol.data, digits = 2)      # Round to 2 decimals 
fval = dot(c, xsol)                       # Optimal cost of reserved capacities

## Solution time of the large deterministic model
println("\nSolution time: ", tend, "\n")

## Print optimal solution
println("Optimal solution:\n")
for i = 1:length(xsol)
    @printf("x[%2d] = %7.2f\n", i, xsol[i])
end

## Print optimal cost of reserved capacities
println("\nOptimal cost of reserved capacities: ", fval)

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:   315000
Number of nonzeros in inequality constraint Jacobian.:   310040
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:   315020
                     variables with only lower bounds:   315020
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:    15000
Total number of inequality constraints...............:    10021
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:    10021

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 


Solution time: 350.6432421207428

Optimal solution:

x[ 1] =    0.00
x[ 2] =   92.00
x[ 3] =    0.00
x[ 4] =   51.00
x[ 5] =    0.00
x[ 6] =    0.00
x[ 7] =   77.00
x[ 8] =    0.00
x[ 9] =   22.00
x[10] =  100.00
x[11] =    0.00
x[12] =    4.00
x[13] =   29.00
x[14] =    0.00
x[15] =    0.00
x[16] =    0.00
x[17] =    0.00
x[18] =    0.00
x[19] =   47.00
x[20] =    0.00

Optimal cost of reserved capacities: 3000.0


In [82]:
#######################################################################################
####           NOTE: RESTART THE KERNEL BEFORE TESTING THE ADMM CODE.             #####  
####                 MEMORY MAY BECOME AN ISSUE OTHERWISE.                        ##### 
#######################################################################################
#### ADMM using Ipopt (other available solvers do not support quadratic objective) ####
#######################################################################################
using Distributed               # For parallel computing 
using Random                    # For controlling randomness
using Printf                    # For C-style printf macro
using SharedArrays              # For shared memory access

addprocs(10)                     # Use 4 parallel threads (add 4 new workers)

@everywhere using LinearAlgebra # For norm() function 
@everywhere using JuMP          # NOTE: The macro @everywhere makes the loaded packages available to all threads
@everywhere using Ipopt         # Solver capable of solving quadratic objectives

In [83]:
## Function to solve a small instance of the deterministic model.
##
## TODO: Fill in the missing code inside the function. Compare your solution 
##       to that of the small deterministic model. These should be almost equal.
##
## Input argument: penalty parameter ρ
function admm_model_small(ρ::Float64)
    
    ##################### PROBLEM DATA FOR SMALLER INSTANCE ########################
    Random.seed!(1)                   # Control random number generation
    nI = 20                           # Number of suppliers
    nJ = 30                           # Number of clients
    nS = 100                          # Number of scenarios
    I = 1:nI                          # Supplier index set
    J = 1:nJ                          # Client index set
    S = 1:nS                          # Scenario index set
    #### Generate random data for the problem
    c = rand(5:20, nI)                # Unit capacity costs of suppliers
    d = rand(nJ,nS).*rand(10:50, nJ)  # Client demands in all scenarios
    q = rand(5000:10000, nJ)          # Unit costs of unfulfilled demand
    p = ones(nS).*1/nS                # Scenario probabilities
    f = rand(3:45, (nI,nJ))           # Unit costs to fulfil demands
    b = rand(20:100, nI)              # Max supplier capacities
    B = 3000                          # Max budget (cost) for capacity acquisition
    ################################################################################

    tini = time()                      # Start timer
    
    ## NOTE: ADMM approach starts from here. Complete the missing code parts that
    ##       are requested. Code lines to be completed are marked as follows:
    ##       TODO: Complete the following expression. (For example)

    ## Problem parameters and memory allocation. NOTE: Compare with Exercise 10.1
    x_s  = SharedArray(zeros(nI, nS))  # Store each x vector in (x,y)-steps of each scenario
    v_s  = SharedArray(zeros(nI, nS))  # Store each v vector in v-steps of each scenario
    z    = SharedArray(zeros(nI))      # Store z vectors at each ADNM iteration
    
    ## Main loop with 200 ADMM iterations
    for k = 1:200
        ## Loop for solving each (x,y) step separately for each scenario s in S
        ## NOTE: Compare with Exercise 10.1
        ## NOTE: The macro @distributed allocates scenario subproblems to available
        ##       threads. The macro @sync ensures that all scenario subproblems
        ##       are solved before proceeding with the code after the for loop
        @sync @distributed for s in S   
            
            ## Model to solve (x,y)-step of the current scenario subproblem s
            scen_m = Model(with_optimizer(Ipopt.Optimizer)) 
            
            @variable(scen_m, x[i in I] >= 0)                    # Reserved capacity variables
            @variable(scen_m, y[i in I, j in J] >= 0)            # Demand fulfilment variables
            @variable(scen_m, u[j in J] >= 0)                    # Unfulfilled demand variables

            @constraint(scen_m, [i in I], x[i] <= b[i])          # Max capacity constraint
            @constraint(scen_m, sum(c[i]*x[i] for i in I) <= B)  # Max capacity budget (cost)
            
            ## Capacity reserve limit constraint for each supplier i
            @constraint(scen_m, [i in I], sum(y[i,j] for j in J) <= x[i])
            
            ## Demand balance constraint (u[j] is unfulfilled demand of client j)
            @constraint(scen_m, [j in J], sum(y[i,j] for i in I) == d[j,s] - u[j])

            #### TODO: Complete this objective to compute (x, y) - step for the
            ####       current scenario. Compare with Exercise 10.1
            @objective(scen_m, Min,
                sum(c[i]*x[i] + v_s[i,s]*x[i] for i in I) + 
                sum(f[i,j]*y[i,j] for i in I, j in J) + 
                sum(q[j]*u[j] for j in J) + 
                #sum((x[i]-z[i]) for i in I) +  # Before penalty?
                (ρ/2.0)*sum((x[i]-z[i])^2 for i in I) # Norm
            )

            ## Redirect stdout to omit unnecessary output from workers
            tmp = stdout           
            redirect_stdout()

            ## Solve the (x,y) step for the current scenario
            optimize!(scen_m)

            ## Restore stdout back to normal 
            redirect_stdout(tmp)  

            ## Store the value of x for the current scenario
            x_s[:,s] = value.(x[:])
        end

        ## Compute primal and dual residuals. We use SharedArray to exploit parallelism
        tol = SharedArray(zeros(nS))
        
        #### TODO: Complete the computation of the residual for each s inside the for loop.
        ####       Compare with Exercise 10.1
        @sync @distributed for s in S
             tol[s] = p[s]*ρ*norm(x_s[:,s] - z)
        end
        
        ## Total residual = sum of subproblem residuals
        tol = sum(tol[s] for s in S)

        ## Print current progress
        println("iteration: ", k ,"/ residual: ", tol)
        
        ## Stopping condition: if primal + dual residual is small enough
        if tol < 1e-1
            ## Stop timer
            tend = time() - tini
            return (c, x_s, tend, k)
        end

        #### TODO: Complete the z-step. Compare with Exercise 10.1
        z = x_s*p # Done

        #### TODO: Complete the v-steps for each scenario s.
        ####       Compare with exercise 10.1
        #@sync for s in S
        #    println("s")
        #    println(size(v_s[:,s]))
        #    println(size(x_s[s]))
        #    println(size(z))
        #    println(v_s[:,s] + ρ*(x_s[:,s] - z))
        #    println("emd")
        #end
        @sync @distributed for s in S
            v_s[:,s] = v_s[:,s] + ρ*(x_s[:, s] - z)# Done v_s should be -1 for s
        end
    end
    
    ## Stop timer
    tend = time() - tini
    
    ## Return solution vector x, costs c, and solution time tend
    return (c, x_s, tend, 200)    
end

admm_model_small (generic function with 1 method)

In [84]:
## NOTE: Run this cell twice, the first run is always slower due to "compilation" (Julia uses JIT compilation)
## Solve the small ADMM model

## Try different values of penalty parameter ρ ∈ [1.0, 100.0]
ρ = 100.0                                   

(c, x_s, tend, k) = admm_model_small(ρ)

xsol = round.(x_s[:,1], digits = 2)     # Get optimal x values (reserved capacities)
fval = dot(c, x_s[:,1])                 # Optimal cost of reserved capacities

## Solution time of ADMM:
println("\nADMM Solution time: ", tend, "\n")

## Print optimal solution
println("Optimal solution:\n")
for i = 1:length(xsol)
    @printf("x[%2d] = %7.2f\n", i, xsol[i])
end

## Print optimal cost of reserved capacities
println("\nCost of reserved capacity: ", fval)

iteration: 1/ residual: 10128.151631679988
iteration: 2/ residual: 3897.0472988761526
iteration: 3/ residual: 2094.366692761772
iteration: 4/ residual: 1357.785693925109
iteration: 5/ residual: 659.8889738082378
iteration: 6/ residual: 579.0866346900767
iteration: 7/ residual: 372.44775312180843
iteration: 8/ residual: 329.8535794275483
iteration: 9/ residual: 238.30283218384054
iteration: 10/ residual: 166.70059096506833
iteration: 11/ residual: 115.57367982101998
iteration: 12/ residual: 59.9671000337686
iteration: 13/ residual: 34.30364386423431
iteration: 14/ residual: 32.9878603570172
iteration: 15/ residual: 11.328065122191084
iteration: 16/ residual: 9.531573608007855
iteration: 17/ residual: 7.899776623998097
iteration: 18/ residual: 7.090028215313696
iteration: 19/ residual: 6.682110300726924
iteration: 20/ residual: 6.2833976993005765
iteration: 21/ residual: 5.964693815624256
iteration: 22/ residual: 5.674379076735991
iteration: 23/ residual: 5.353801278021653
iteration: 24/

In [85]:
## Function to solve a large instance of the deterministic model.
##
## TODO: Fill in the missing code inside the function. Compare your solution 
##       to that of the large deterministic model. These should be almost equal.
##
## Input argument: penalty parameter ρ
function admm_model_large(ρ::Float64)
    
    ##################### PROBLEM DATA FOR LARGER INSTANCE #########################
    Random.seed!(1)                   # Control random number generation
    nI = 20                           # Number of suppliers
    nJ = 30                           # Number of clients
    nS = 500                          # Number of scenarios
    I = 1:nI                          # Supplier index set
    J = 1:nJ                          # Client index set
    S = 1:nS                          # Scenario index set
    #### Generate random data for the problem
    c = rand(5:20, nI)                # Unit capacity costs of suppliers
    d = rand(nJ,nS).*rand(10:50, nJ)  # Client demands in all scenarios
    q = rand(5000:10000, nJ)          # Unit costs of unfulfilled demand
    p = ones(nS).*1/nS                # Scenario probabilities
    f = rand(3:45, (nI,nJ))           # Unit costs to fulfil demands
    b = rand(20:100, nI)              # Max supplier capacities
    B = 3000                          # Max budget (cost) for capacity acquisition
    ################################################################################

    tini = time()                      # Start timer
    
    ## NOTE: ADMM approach starts from here. Complete the missing code parts that
    ##       are requested. Code lines to be completed are marked as follows:
    ##       TODO: Complete the following expression. (For example)

    ## Problem parameters and memory allocation. NOTE: Compare with Exercise 10.1
    x_s  = SharedArray(zeros(nI, nS))  # Store each x vector in (x,y)-steps of each scenario
    v_s  = SharedArray(zeros(nI, nS))  # Store each v vector in v-steps of each scenario
    z    = SharedArray(zeros(nI))      # Store z vectors at each ADNM iteration
    
    ## Main loop with 200 ADMM iterations
    for k = 1:200
        ## Loop for solving each (x,y) step separately for each scenario s in S
        ## NOTE: Compare with Exercise 10.1
        ## NOTE: The macro @distributed allocates scenario subproblems to available
        ##       threads. The macro @sync ensures that all scenario subproblems
        ##       are solved before proceeding with the code after the for loop
        @sync @distributed for s in S   
            
            ## Model to solve (x,y)-step of the current scenario subproblem s
            scen_m = Model(with_optimizer(Ipopt.Optimizer)) 
            
            @variable(scen_m, x[i in I] >= 0)                    # Reserved capacity variables
            @variable(scen_m, y[i in I, j in J] >= 0)            # Demand fulfilment variables
            @variable(scen_m, u[j in J] >= 0)                    # Unfulfilled demand variables

            @constraint(scen_m, [i in I], x[i] <= b[i])          # Max capacity constraint
            @constraint(scen_m, sum(c[i]*x[i] for i in I) <= B)  # Max capacity budget (cost)
            
            ## Capacity reserve limit constraint for each supplier i
            @constraint(scen_m, [i in I], sum(y[i,j] for j in J) <= x[i])
            
            ## Demand balance constraint (u[j] is unfulfilled demand of client j)
            @constraint(scen_m, [j in J], sum(y[i,j] for i in I) == d[j,s] - u[j])

            #### TODO: Complete this objective to compute (x, y) - step for the
            ####       current scenario. Compare with Exercise 10.1
            @objective(scen_m, Min,
                sum(c[i]*x[i] + v_s[i,s]*x[i] for i in I) + 
                sum(f[i,j]*y[i,j] for i in I, j in J) + 
                sum(q[j]*u[j] for j in J) + 
                #sum((x[i]-z[i]) for i in I) +  # Before penalty?
                (ρ/2.0)*sum((x[i]-z[i])^2 for i in I) # Norm
            )

            ## Redirect stdout to omit unnecessary output from workers
            tmp = stdout           
            redirect_stdout()

            ## Solve the (x,y) step for the current scenario
            optimize!(scen_m)

            ## Restore stdout back to normal 
            redirect_stdout(tmp)  

            ## Store the value of x for the current scenario
            x_s[:,s] = value.(x[:])
        end

        ## Compute primal and dual residuals. We use array to exploit parallelism
        tol = SharedArray(zeros(nS))
        
        #### TODO: Complete the computation of the residual for each s inside the for loop.
        ####       Compare with Exercise 10.1
        @sync @distributed for s in S
             tol[s] = p[s]*ρ*norm(x_s[:,s] - z)
        end
        
        ## Total residual = sum of subproblem residuals
        tol = sum(tol[s] for s in S)

        ## Print current progress
        println("iteration: ", k ,"/ residual: ", tol)
        
        ## Stopping condition: if primal + dual residual is small enough
        if tol < 1e-1
            ## Stop timer
            tend = time() - tini
            return (c, x_s, tend, k)
        end

        #### TODO: Complete the z-step. Compare with Exercise 10.1
        z = sum(p[s]*x_s[:,s] for s in S)

        #### TODO: Complete the v-steps for each scenario s.
        ####       Compare with exercise 10.1
        @sync @distributed for s in S
            v_s[:,s] = v_s[:,s] + ρ*(x_s[:, s] - z)
        end
    end
    
    ## Stop timer
    tend = time() - tini
    
    ## Return solution vector x, costs c, and solution time tend
    return (c, x_s, tend, 200)    
end

admm_model_large (generic function with 1 method)

In [86]:
## NOTE: Run this cell twice, the first run is always slower due to "compilation" (Julia uses JIT compilation)
## Solve the large ADMM model

## Try different values of penalty parameter ρ ∈ [1.0, 100.0]
ρ = 100.0                                   

(c, x_s, tend, k) = admm_model_large(ρ)

xsol = round.(x_s[:,1], digits = 2)     # Get optimal x values (reserved capacities)
fval = dot(c, x_s[:,1])                 # Optimal cost of reserved capacities

## Solution time of ADMM:
println("\nADMM Solution time: ", tend, "\n")

## Print optimal solution
println("Optimal solution:\n")
for i = 1:length(xsol)
    @printf("x[%2d] = %7.2f\n", i, xsol[i])
end

## Print optimal cost of reserved capacities
println("\nCost of reserved capacity: ", fval)

iteration: 1/ residual: 9713.373880009747
iteration: 2/ residual: 3847.767198231959
iteration: 3/ residual: 1889.8368339528315
iteration: 4/ residual: 1378.3977762653317
iteration: 5/ residual: 992.2924574159562
iteration: 6/ residual: 841.9114691101751
iteration: 7/ residual: 745.1075741178236
iteration: 8/ residual: 713.1971163543981
iteration: 9/ residual: 496.44442169031777
iteration: 10/ residual: 371.23657423415443
iteration: 11/ residual: 227.41513299183174
iteration: 12/ residual: 216.13875069724313
iteration: 13/ residual: 212.61689564343087
iteration: 14/ residual: 211.02641580538906
iteration: 15/ residual: 209.60974383999866
iteration: 16/ residual: 208.8142369153094
iteration: 17/ residual: 207.79105082062785
iteration: 18/ residual: 207.13265773674212
iteration: 19/ residual: 206.56270346338883
iteration: 20/ residual: 205.87012907799908
iteration: 21/ residual: 205.17438022765512
iteration: 22/ residual: 204.07185214416552
iteration: 23/ residual: 203.64823308845203
iter

In [87]:
using Plots

In [88]:
plotly() # or pyplot()

Plots.PlotlyBackend()

In [74]:
max = convert(Float64, 100.0)
ρs = collect(LinRange(1, 10, 10))
ks = zeros(0)

ρs2 = collect(LinRange(15, 100, 15))
ks2 = zeros(0)

for ρ = ρs
    println("Starting model $ρ")
    (c, x_s, tend, k) = admm_model_large(convert(Float64, ρ)) # admm_model_large(ρ)
    append!(ks, k) # change k to tend to plot execution time
end


#savefig("performance_plot.png") # to save figure

Starting model 1.0
iteration: 1/ residual: 148.1213542867282
iteration: 2/ residual: 37.932543459163725
iteration: 3/ residual: 8.84922753356021
iteration: 4/ residual: 3.1624274973850754
iteration: 5/ residual: 2.4624284756212655
iteration: 6/ residual: 2.156376054261021
iteration: 7/ residual: 2.073837462733628
iteration: 8/ residual: 2.0160879469313984
iteration: 9/ residual: 1.9275826540389747
iteration: 10/ residual: 1.8216550940950762
iteration: 11/ residual: 1.5030162005976235
iteration: 12/ residual: 1.2090265089623315
iteration: 13/ residual: 0.889527810133315
iteration: 14/ residual: 0.6144964498543098
iteration: 15/ residual: 0.30720017383105663
iteration: 16/ residual: 0.20634287488424236
iteration: 17/ residual: 0.14953014960429334
iteration: 18/ residual: 0.11909837649446606
iteration: 19/ residual: 0.09512096304413803
Starting model 2.0
iteration: 1/ residual: 295.88666415235207
iteration: 2/ residual: 78.81943003017774
iteration: 3/ residual: 15.55007100023877
iteration

iteration: 31/ residual: 0.1536753702501121
iteration: 32/ residual: 0.13766093880582525
iteration: 33/ residual: 0.1278584660031196
iteration: 34/ residual: 0.1252536980686823
iteration: 35/ residual: 0.12237755702583786
iteration: 36/ residual: 0.11641004244850749
iteration: 37/ residual: 0.10541363727846956
iteration: 38/ residual: 0.10026875516323648
iteration: 39/ residual: 0.0919686827588664
Starting model 9.0
iteration: 1/ residual: 1299.6641224754133
iteration: 2/ residual: 323.1465593944766
iteration: 3/ residual: 177.31150868408525
iteration: 4/ residual: 34.120718523535885
iteration: 5/ residual: 20.29531225889487
iteration: 6/ residual: 18.70911679916898
iteration: 7/ residual: 17.53578253454944
iteration: 8/ residual: 16.592738597194085
iteration: 9/ residual: 15.26645055500715
iteration: 10/ residual: 9.389706570317827
iteration: 11/ residual: 8.326413631354086
iteration: 12/ residual: 3.3783345468699766
iteration: 13/ residual: 1.813058560764804
iteration: 14/ residual: 

LoadError: syntax: extra token ")" after end of expression

In [75]:
plot(ρs, ks, 
    linewidth = 3,
    xlabel = "ρ",
    ylabel = "Iterations",
    #title  = "Penalty effect on number of iterations",
    size   = (600,400),
    reuse  = false,
    tickfontsize   = 8,
    legend = false,
    legendfontsize = 10,
    guidefontsize  = 10,
    grid = true)
scatter!(ρs, ks,show=true)

<!DOCTYPE html>
 
 
 Plots.jl

In [76]:
for ρ = ρs2
    println("Starting model $ρ")
    (c, x_s, tend, k) = admm_model_large(convert(Float64, ρ)) # admm_model_large(ρ)
    append!(ks2, k) # change k to tend to plot execution time
end
plot!(cat(ρs,ρs2, dims=1), cat(ks, ks2, dims=1), 
    linewidth = 3,
    xlabel = "ρ",
    ylabel = "Iterations",
    #title  = "Penalty effect on number of iterations",
    size   = (600,400),
    reuse  = false,
    tickfontsize   = 8,
    legend = false,
    legendfontsize = 10,
    guidefontsize  = 10,
    grid = true)
scatter!(cat(ρs,ρs2, dims=1), cat(ks, ks2, dims=1),show=true)

Opening in existing browser session.
Starting model 15.0
iteration: 1/ residual: 2132.78384601
iteration: 2/ residual: 468.8907946137091
iteration: 3/ residual: 223.60418820516418
iteration: 4/ residual: 207.27673892914086
iteration: 5/ residual: 159.4480833546063
iteration: 6/ residual: 34.390331951992174
iteration: 7/ residual: 31.504088818711264
iteration: 8/ residual: 27.497679033745005
iteration: 9/ residual: 20.188816656597282
iteration: 10/ residual: 14.647006207617464
iteration: 11/ residual: 9.445212044901854
iteration: 12/ residual: 2.832268146084213
iteration: 13/ residual: 2.2211856423076655
iteration: 14/ residual: 1.6111572988988998
iteration: 15/ residual: 1.330479582805843
iteration: 16/ residual: 1.1649618713261405
iteration: 17/ residual: 1.0423298744175644
iteration: 18/ residual: 0.935472793332212
iteration: 19/ residual: 0.8580342507157479
iteration: 20/ residual: 0.7695536291064673
iteration: 21/ residual: 0.692872552272365
iteration: 22/ residual: 0.6194153178265

iteration: 4/ residual: 449.03256160458
iteration: 5/ residual: 228.18496603857795
iteration: 6/ residual: 214.74005576897537
iteration: 7/ residual: 210.75634723158615
iteration: 8/ residual: 208.37381154955062
iteration: 9/ residual: 205.53914112330375
iteration: 10/ residual: 202.5782421974814
iteration: 11/ residual: 199.68656185189164
iteration: 12/ residual: 197.20703414970276
iteration: 13/ residual: 96.0498625953958
iteration: 14/ residual: 38.62364016994384
iteration: 15/ residual: 29.700182265630538
iteration: 16/ residual: 19.535663542614714
iteration: 17/ residual: 9.716690156665136
iteration: 18/ residual: 4.086005085360279
iteration: 19/ residual: 3.225877051254407
iteration: 20/ residual: 2.592069404422623
iteration: 21/ residual: 2.2075082018915055
iteration: 22/ residual: 1.934360685748193
iteration: 23/ residual: 1.661239879251896
iteration: 24/ residual: 1.4505612055616068
iteration: 25/ residual: 1.3486302897056488
iteration: 26/ residual: 1.2754307250703583
iterati

iteration: 44/ residual: 0.358782849947004
iteration: 45/ residual: 0.3246187758508775
iteration: 46/ residual: 0.3162816822699993
iteration: 47/ residual: 0.3140193500182787
iteration: 48/ residual: 0.31152745877723276
iteration: 49/ residual: 0.31086131052172467
iteration: 50/ residual: 0.3061121907647978
iteration: 51/ residual: 0.2983995845994162
iteration: 52/ residual: 0.2891982840566708
iteration: 53/ residual: 0.2629453688173555
iteration: 54/ residual: 0.23127045158457657
iteration: 55/ residual: 0.20748983259539125
iteration: 56/ residual: 0.19718727685720333
iteration: 57/ residual: 0.1964457894135644
iteration: 58/ residual: 0.18018372855777254
iteration: 59/ residual: 0.1523172316758396
iteration: 60/ residual: 0.13092934362293868
iteration: 61/ residual: 0.12299770848268006
iteration: 62/ residual: 0.12137589556917529
iteration: 63/ residual: 0.11816736678648385
iteration: 64/ residual: 0.11575697801131474
iteration: 65/ residual: 0.1147005788228811
iteration: 66/ residua

iteration: 57/ residual: 0.1623264521883023
iteration: 58/ residual: 0.16074187663365502
iteration: 59/ residual: 0.15773039222813853
iteration: 60/ residual: 0.15317896015272267
iteration: 61/ residual: 0.15085927044392733
iteration: 62/ residual: 0.15061450627088876
iteration: 63/ residual: 0.1507457906841674
iteration: 64/ residual: 0.1507784661015811
iteration: 65/ residual: 0.15081759828745614
iteration: 66/ residual: 0.15081117008464898
iteration: 67/ residual: 0.15060405803771293
iteration: 68/ residual: 0.1484561922521626
iteration: 69/ residual: 0.143854291143979
iteration: 70/ residual: 0.1053563548864994
iteration: 71/ residual: 0.039700118921023035
Starting model 81.78571428571428
iteration: 1/ residual: 8725.44200887262
iteration: 2/ residual: 2724.7777269488215
iteration: 3/ residual: 1682.0835858142996
iteration: 4/ residual: 1027.9172312390313
iteration: 5/ residual: 844.6662476520238
iteration: 6/ residual: 745.5930386045561
iteration: 7/ residual: 669.3714590884158
it

iteration: 74/ residual: 0.16631763812298453
iteration: 75/ residual: 0.13298717193162538
iteration: 76/ residual: 0.06682112680980079
Starting model 100.0
iteration: 1/ residual: 9713.373880009747
iteration: 2/ residual: 3847.767198231959
iteration: 3/ residual: 1889.8368339528315
iteration: 4/ residual: 1378.3977762653317
iteration: 5/ residual: 992.2924574159562
iteration: 6/ residual: 841.9114691101751
iteration: 7/ residual: 745.1075741178236
iteration: 8/ residual: 713.1971163543981
iteration: 9/ residual: 496.44442169031777
iteration: 10/ residual: 371.23657423415443
iteration: 11/ residual: 227.41513299183174
iteration: 12/ residual: 216.13875069724313
iteration: 13/ residual: 212.61689564343087
iteration: 14/ residual: 211.02641580538906
iteration: 15/ residual: 209.60974383999866
iteration: 16/ residual: 208.8142369153094
iteration: 17/ residual: 207.79105082062785
iteration: 18/ residual: 207.13265773674212
iteration: 19/ residual: 206.56270346338883
iteration: 20/ residual:

<!DOCTYPE html>
 
 
 Plots.jl

In [12]:
## Kill threads
rmprocs(procs())

┌ Warning: rmprocs: process 1 not removed
└ @ Distributed /buildworker/worker/package_linux32/build/usr/share/julia/stdlib/v1.0/Distributed/src/cluster.jl:932


Task (done) @0xd0fea590